# ICD-9 and CUI

### 2023-07-01

ICD-9 codes downloaded from https://www.cms.gov/medicare/coordination-benefits-recovery-overview/icd-code-lists

In [2]:
# import data
import pandas as pd

# Read the Excel file
excel_file = r'C:\Users\sungh\icd9.xlsx'
icd9 = pd.read_excel(excel_file)

# Convert to CSV
icd9file = r'C:\Users\sungh\icd10.csv'
icd9.to_csv(icd9file, index=False)

In [3]:
# Specify the new column names
new_column_names = {'CODE': 'ICD9', 'LONG DESCRIPTION (VALID ICD-9 FY2023)': 
                    'Description'}

# Rename the columns in the DataFrame
icd9.rename(columns=new_column_names, inplace=True)

#### Drop unnecessary columns:

In [4]:
# Specify the columns to keep
columns_to_keep = ['ICD9', 'Description']

# Create a new DataFrame with only the desired columns
icd9_updated = icd9[columns_to_keep]

# Print the modified DataFrame
print(icd9_updated.head(11))

     ICD9                                  Description
0    0010               Cholera due to vibrio cholerae
1    0011        Cholera due to vibrio cholerae el tor
2    0019                         Cholera, unspecified
3    0020                                Typhoid fever
4    0021                          Paratyphoid fever A
5    0022                          Paratyphoid fever B
6    0023                          Paratyphoid fever C
7    0029               Paratyphoid fever, unspecified
8    0030                   Salmonella gastroenteritis
9    0031                        Salmonella septicemia
10  00320  Localized salmonella infection, unspecified


### Number of ICD-9 Codes: 

In [5]:
# Count the number of rows
num_rows = len(icd9_updated)
print("Number of ICD-9 Codes:", num_rows)

Number of ICD-9 Codes: 13521


#### Add decimal places to format ICD-9:

In [7]:
def add_decimal(icd_code):
    if len(icd_code) > 3:
        return icd_code[:3] + '.' + icd_code[3:]
    else:
        return icd_code

# Apply the add_decimal function to the 'ICD9' column in your dataframe
icd9_updated['ICD9_WITH_DECIMAL'] = icd9_updated['ICD9'].apply(add_decimal)

In [9]:
# Create a new DataFrame without the 'icd9' column
final_icd9 = icd9_updated.drop('ICD9', axis=1)

# Verify the new DataFrame
print(final_icd9.head())

                             Description ICD9_WITH_DECIMAL
0         Cholera due to vibrio cholerae             001.0
1  Cholera due to vibrio cholerae el tor             001.1
2                   Cholera, unspecified             001.9
3                          Typhoid fever             002.0
4                    Paratyphoid fever A             002.1


### From https://documentation.uts.nlm.nih.gov/scripts/crosswalk.py

### 06-27 Update: This code below works (from: https://documentation.uts.nlm.nih.gov/scripts/search-terms.py)

### This step will process the entire data, using "Description" column to produce CSV file that matches CUI: 
#### (will take a while)

In [10]:
import requests
import pandas as pd
from math import ceil

def retrieve_cui(apikey, version, icd_codes, batch_size):
    uri = "https://uts-ws.nlm.nih.gov"
    content_endpoint = "/rest/search/" + version
    full_url = uri + content_endpoint
    result_list = []

    total_batches = ceil(len(icd_codes) / batch_size)

    for batch in range(total_batches):
        start_index = batch * batch_size
        end_index = (batch + 1) * batch_size
        batch_codes = icd_codes[start_index:end_index]

        try:
            for icd_code in batch_codes:
                page = 0  # Reset the page counter for each ICD code
                while True:
                    page += 1
                    query = {'string': icd_code, 'apiKey': apikey, 'pageNumber': page}
                    r = requests.get(full_url, params=query)
                    r.raise_for_status()
                    r.encoding = 'utf-8'
                    outputs = r.json()

                    items = outputs['result']['results']

                    if len(items) == 0:
                        if page == 1:
                            print('No results found for ICD code: ' + icd_code + '\n')
                            break
                        else:
                            break

                    print("Results for ICD code: " + icd_code + ", page " + str(page) + "\n")

                    for result in items:
                        cui = result['ui']
                        print('ICD-9: ' + icd_code)
                        print('CUI: ' + cui)
                        print('\n')

                        result_dict = {
                            'ICD-9': icd_code,
                            'CUI': cui,
                        }
                        result_list.append(result_dict)

                    print('*********')

        except Exception as except_error:
            print(except_error)

    # Convert the result_list to a DataFrame
    result_df = pd.DataFrame(result_list, columns=['ICD-9', 'CUI'])

    # Save the DataFrame to a CSV file
    result_df.to_csv('ICD9_CUI_FINAL.csv', index=False)
    print("CSV file saved successfully.")

# Set your UMLS API key and version here
apikey = "49182007-9896-41a5-aae6-f2cb70d0ca5e"
version = "current"

# Replace with your actual ICD codes dataframe
icd_codes = final_icd9['ICD9_WITH_DECIMAL']
batch_size = 200  # Number of codes to process in each batch

# Call the retrieve_cui function with the provided API key, version, icd_codes, and batch_size
retrieve_cui(apikey, version, icd_codes, batch_size)

Results for ICD code: 001.0, page 1

ICD-9: 001.0
CUI: C0008354


*********
Results for ICD code: 001.1, page 1

ICD-9: 001.1
CUI: C0343372


*********
Results for ICD code: 001.9, page 1

ICD-9: 001.9
CUI: C0008354


*********
Results for ICD code: 002.0, page 1

ICD-9: 002.0
CUI: C0041466


*********
Results for ICD code: 002.1, page 1

ICD-9: 002.1
CUI: C0343375


ICD-9: 002.1
CUI: C4279561


*********
Results for ICD code: 002.2, page 1

ICD-9: 002.2
CUI: C0343376


*********
Results for ICD code: 002.3, page 1

ICD-9: 002.3
CUI: C0343377


*********
Results for ICD code: 002.9, page 1

ICD-9: 002.9
CUI: C0030528


*********
Results for ICD code: 003.0, page 1

ICD-9: 003.0
CUI: C0036114


*********
Results for ICD code: 003.1, page 1

ICD-9: 003.1
CUI: C0152486


*********
Results for ICD code: 003.20, page 1

ICD-9: 003.20
CUI: C0152487


*********
Results for ICD code: 003.21, page 1

ICD-9: 003.21
CUI: C0152488


*********
Results for ICD code: 003.22, page 1

ICD-9: 003.22
CUI

Results for ICD code: 013.21, page 1

ICD-9: 013.21
CUI: C0152669


*********
Results for ICD code: 013.22, page 1

ICD-9: 013.22
CUI: C0152670


*********
Results for ICD code: 013.23, page 1

ICD-9: 013.23
CUI: C0152671


*********
Results for ICD code: 013.24, page 1

ICD-9: 013.24
CUI: C0152672


*********
Results for ICD code: 013.25, page 1

ICD-9: 013.25
CUI: C0152673


*********
Results for ICD code: 013.26, page 1

ICD-9: 013.26
CUI: C0152674


*********
Results for ICD code: 013.30, page 1

ICD-9: 013.30
CUI: C0374958


*********
Results for ICD code: 013.31, page 1

ICD-9: 013.31
CUI: C0152676


*********
Results for ICD code: 013.32, page 1

ICD-9: 013.32
CUI: C0152677


*********
Results for ICD code: 013.33, page 1

ICD-9: 013.33
CUI: C0152678


*********
Results for ICD code: 013.34, page 1

ICD-9: 013.34
CUI: C0152679


*********
Results for ICD code: 013.35, page 1

ICD-9: 013.35
CUI: C0152680


*********
Results for ICD code: 013.36, page 1

ICD-9: 013.36
CUI: C015268

Results for ICD code: 015.82, page 1

ICD-9: 015.82
CUI: C0152774


*********
Results for ICD code: 015.83, page 1

ICD-9: 015.83
CUI: C0152775


*********
Results for ICD code: 015.84, page 1

ICD-9: 015.84
CUI: C0152776


*********
Results for ICD code: 015.85, page 1

ICD-9: 015.85
CUI: C0152777


*********
Results for ICD code: 015.86, page 1

ICD-9: 015.86
CUI: C0152778


*********
Results for ICD code: 015.90, page 1

ICD-9: 015.90
CUI: C0374968


*********
Results for ICD code: 015.91, page 1

ICD-9: 015.91
CUI: C0152780


*********
Results for ICD code: 015.92, page 1

ICD-9: 015.92
CUI: C0152781


*********
Results for ICD code: 015.93, page 1

ICD-9: 015.93
CUI: C0152782


*********
Results for ICD code: 015.94, page 1

ICD-9: 015.94
CUI: C0152783


*********
Results for ICD code: 015.95, page 1

ICD-9: 015.95
CUI: C0152784


*********
Results for ICD code: 015.96, page 1

ICD-9: 015.96
CUI: C0152785


*********
Results for ICD code: 016.00, page 1

ICD-9: 016.00
CUI: C004132

Results for ICD code: 017.43, page 1

ICD-9: 017.43
CUI: C0152877


*********
Results for ICD code: 017.44, page 1

ICD-9: 017.44
CUI: C0152878


*********
Results for ICD code: 017.45, page 1

ICD-9: 017.45
CUI: C0152879


*********
Results for ICD code: 017.46, page 1

ICD-9: 017.46
CUI: C0152880


*********
Results for ICD code: 017.50, page 1

ICD-9: 017.50
CUI: C0152882


*********
Results for ICD code: 017.51, page 1

ICD-9: 017.51
CUI: C0152883


*********
Results for ICD code: 017.52, page 1

ICD-9: 017.52
CUI: C0152884


*********
Results for ICD code: 017.53, page 1

ICD-9: 017.53
CUI: C0152885


*********
Results for ICD code: 017.54, page 1

ICD-9: 017.54
CUI: C0152886


*********
Results for ICD code: 017.55, page 1

ICD-9: 017.55
CUI: C0152887


*********
Results for ICD code: 017.56, page 1

ICD-9: 017.56
CUI: C0152888


*********
Results for ICD code: 017.60, page 1

ICD-9: 017.60
CUI: C0152889


*********
Results for ICD code: 017.61, page 1

ICD-9: 017.61
CUI: C015289

Results for ICD code: 024, page 3

ICD-9: 024
CUI: C5390543


ICD-9: 024
CUI: C5390541


ICD-9: 024
CUI: C3497823


ICD-9: 024
CUI: C4056932


ICD-9: 024
CUI: C2949724


ICD-9: 024
CUI: C4293140


ICD-9: 024
CUI: C3817827


ICD-9: 024
CUI: C3855299


ICD-9: 024
CUI: C4059886


ICD-9: 024
CUI: C5771245


ICD-9: 024
CUI: C5706030


ICD-9: 024
CUI: C5771242


ICD-9: 024
CUI: C3200734


ICD-9: 024
CUI: C2188757


ICD-9: 024
CUI: C2736787


ICD-9: 024
CUI: C5148488


ICD-9: 024
CUI: C2735382


*********
Results for ICD code: 025, page 1

ICD-9: 025
CUI: C2493584


ICD-9: 025
CUI: C0025229


ICD-9: 025
CUI: C3886673


ICD-9: 025
CUI: C4721708


ICD-9: 025
CUI: C0133776


ICD-9: 025
CUI: C5543863


ICD-9: 025
CUI: C0767465


ICD-9: 025
CUI: C5552872


ICD-9: 025
CUI: C4287684


ICD-9: 025
CUI: C0527671


ICD-9: 025
CUI: C1567951


ICD-9: 025
CUI: C5667046


ICD-9: 025
CUI: C3712551


ICD-9: 025
CUI: C2934228


ICD-9: 025
CUI: C3827040


ICD-9: 025
CUI: C5544575


ICD-9: 025
CUI: C0024966


IC

...

No results found for ICD code: E82.78

No results found for ICD code: E82.79

No results found for ICD code: E82.80

No results found for ICD code: E82.82

No results found for ICD code: E82.84

No results found for ICD code: E82.88

No results found for ICD code: E82.89

No results found for ICD code: E82.90

No results found for ICD code: E82.94

No results found for ICD code: E82.98

No results found for ICD code: E82.99

Results for ICD code: E83.00, page 1

ICD-9: E83.00
CUI: C0012714


*********
Results for ICD code: E83.01, page 1

ICD-9: E83.01
CUI: C0019202


*********
No results found for ICD code: E83.02

No results found for ICD code: E83.03

No results found for ICD code: E83.04

No results found for ICD code: E83.05

No results found for ICD code: E83.06

No results found for ICD code: E83.07

No results found for ICD code: E83.08

Results for ICD code: E83.09, page 1

ICD-9: E83.09
CUI: C2874298


ICD-9: E83.09
CUI: C2874297


*********
Results for ICD code: E83.10, page

No results found for ICD code: E85.43

No results found for ICD code: E85.48

No results found for ICD code: E85.50

No results found for ICD code: E85.51

No results found for ICD code: E85.52

No results found for ICD code: E85.53

No results found for ICD code: E85.54

No results found for ICD code: E85.55

No results found for ICD code: E85.56

No results found for ICD code: E85.58

No results found for ICD code: E85.59

No results found for ICD code: E85.6

No results found for ICD code: E85.7

No results found for ICD code: E85.80

Results for ICD code: E85.81, page 1

ICD-9: E85.81
CUI: C4509023


*********
Results for ICD code: E85.82, page 1

ICD-9: E85.82
CUI: C4480969


ICD-9: E85.82
CUI: C4509024


*********
No results found for ICD code: E85.83

No results found for ICD code: E85.84

No results found for ICD code: E85.85

No results found for ICD code: E85.86

No results found for ICD code: E85.87

No results found for ICD code: E85.88

Results for ICD code: E85.89, page 1

No results found for ICD code: E88.51

No results found for ICD code: E88.52

No results found for ICD code: E88.53

No results found for ICD code: E88.54

No results found for ICD code: E88.59

No results found for ICD code: E88.60

No results found for ICD code: E88.69

No results found for ICD code: E88.7

No results found for ICD code: E88.80

Results for ICD code: E88.81, page 1

ICD-9: E88.81
CUI: C0524620


*********
No results found for ICD code: E88.88

Results for ICD code: E88.89, page 1

ICD-9: E88.89
CUI: C1386301


ICD-9: E88.89
CUI: C0494362


*********
No results found for ICD code: E89.00

No results found for ICD code: E89.01

No results found for ICD code: E89.02

No results found for ICD code: E89.03

No results found for ICD code: E89.08

No results found for ICD code: E89.09

No results found for ICD code: E89.10

No results found for ICD code: E89.11

No results found for ICD code: E89.12

No results found for ICD code: E89.13

No results found for ICD code: E89.

No results found for ICD code: E93.06

No results found for ICD code: E93.07

No results found for ICD code: E93.08

No results found for ICD code: E93.09

No results found for ICD code: E93.10

No results found for ICD code: E93.11

No results found for ICD code: E93.12

No results found for ICD code: E93.13

No results found for ICD code: E93.14

No results found for ICD code: E93.15

No results found for ICD code: E93.16

No results found for ICD code: E93.17

No results found for ICD code: E93.18

No results found for ICD code: E93.19

No results found for ICD code: E93.20

No results found for ICD code: E93.21

No results found for ICD code: E93.22

No results found for ICD code: E93.23

No results found for ICD code: E93.24

No results found for ICD code: E93.25

No results found for ICD code: E93.26

No results found for ICD code: E93.27

No results found for ICD code: E93.28

No results found for ICD code: E93.29

No results found for ICD code: E93.30

No results found for ICD 

No results found for ICD code: E95.88

No results found for ICD code: E95.89

No results found for ICD code: E95.9

No results found for ICD code: E96.00

No results found for ICD code: E96.01

No results found for ICD code: E96.1

No results found for ICD code: E96.20

No results found for ICD code: E96.21

No results found for ICD code: E96.22

No results found for ICD code: E96.29

No results found for ICD code: E96.3

No results found for ICD code: E96.4

No results found for ICD code: E96.50

No results found for ICD code: E96.51

No results found for ICD code: E96.52

No results found for ICD code: E96.53

No results found for ICD code: E96.54

No results found for ICD code: E96.55

No results found for ICD code: E96.56

No results found for ICD code: E96.57

No results found for ICD code: E96.58

No results found for ICD code: E96.59

No results found for ICD code: E96.6

No results found for ICD code: E96.70

No results found for ICD code: E96.71

No results found for ICD code: